# Assignment 1
In this assignment you will be creating tools for learning and testing language models.
The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.


*As a preparation for this task, download the data files from the course git repository.

The relevant files are under **lm-languages-data-new**:


*   en.csv (or the equivalent JSON file)
*   es.csv (or the equivalent JSON file)
*   fr.csv (or the equivalent JSON file)
*   in.csv (or the equivalent JSON file)
*   it.csv (or the equivalent JSON file)
*   nl.csv (or the equivalent JSON file)
*   pt.csv (or the equivalent JSON file)
*   tl.csv (or the equivalent JSON file)
*   test.csv (or the equivalent JSON file)





In [42]:
# imports
import pandas as pd
import numpy as np
from itertools import combinations
from collections import Counter
import time
import glob
import os 
from sklearn.metrics import f1_score
from IPython.display import display


In [43]:
!git clone https://github.com/kfirbar/nlp-course.git

fatal: destination path 'nlp-course' already exists and is not an empty directory.




---



**Important note: please use only the files under lm-languages-data-new and NOT under lm-languages-data**


---



In [44]:

!ls nlp-course/lm-languages-data-new


'ls' is not recognized as an internal or external command,
operable program or batch file.


In [45]:
data_files = {'en_df': 'en.csv',
              'es_df': 'es.csv',
              'fr_df': 'fr.csv',
              'in_df': 'in.csv',
              'it_df': 'it.csv',
              'nl_df': 'nl.csv',
              'pt_df': 'pt.csv',
              'tl_df': 'tl.csv'}

    
directory = 'nlp-course/lm-languages-data-new/'    
for (key, value) in data_files.items():
    data_files[key] = directory + value
    
languages_list = list(data_files.keys())

start_token = '↠'
end_token = '↞'

**Part 1**

Write a function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. **Our token definition is a single UTF-8 encoded character**. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

In [46]:
def preprocess(data_files):
    """
    data frame is table from 2 columns:
        1. tweet id
        2. tweet text
    """  
    tokens = []
    for path in data_files.values():
        df = pd.read_csv(path)
        if tokens.__len__() == 0 :
            columns_list = df.columns.to_list()
        for text in df[columns_list[-1]].values:
            tokens.extend(list(text))
    return list(set(tokens))

In [47]:
vocabulary = preprocess(data_files)

**Part 2**

Write a function lm that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant n-1 sequences, and the values are dictionaries with the n_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{
  "ab":{"c":0.5, "b":0.25, "d":0.25},
  "ca":{"a":0.2, "b":0.7, "d":0.1}
}

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [48]:
#helper function
def tweets_to_text(data_file_path, n):
    """
    data frame is table from 2 columns:
        1. tweet id
        2. tweet text
    """
    df = pd.read_csv(r''+ data_file_path)
    debug = False
    if debug == True:
        df = df[0:100]
    columns_list = df.columns.to_list()
    tweets_list = df[columns_list[-1]].apply(lambda x: start_token + x + end_token).values
    text = ''.join(tweets_list)
    
    text = start_token * (n-1) + text + end_token * (n-1)

    return text
def reorder_list(List, index_list):
    return [List[i] for i in index_list]



In [49]:
def lm(n, vocabulary, data_file_path, add_one):
    # n - the n-gram to use (e.g., 1 - unigram, 2 - bigram, etc.)
    # vocabulary - the vocabulary list (which you should use for calculating add_one smoothing)
    # data_file_path - the data_file from which we record probabilities for our model
    # add_one - True/False (use add_one smoothing or not)
  
    lm_dict = {}
    V = len(vocabulary)

    text = tweets_to_text(data_file_path, n)

    # Extract n - 1 length substrings
    n_1_gram = [text[i: i + n-1] for i in range(len(text) - (n-1))]
    counter_obj_n_1_gram = dict(Counter(n_1_gram))

    # Extract n length substrings
    n_gram = [text[i: i + n] for i in range(len(text) - n)]
    counter_obj_n_gram = dict(Counter(n_gram))

    for key in counter_obj_n_1_gram.keys():
        inner_dict = {}
        if add_one:
            gen = (key_1 for key_1 in counter_obj_n_gram.keys() if key_1[0:n-1] == key)
            for key_1 in gen:
                val = (int(counter_obj_n_gram[key_1]) + 1) / (int(counter_obj_n_1_gram[key]) + V)
                inner_dict[key_1[-1]] = val

            gen = (token for token in vocabulary if not(token in inner_dict))
            for key_1 in gen:
                val = 1 /  (int(counter_obj_n_1_gram[key]) + V)
                inner_dict[key_1[-1]] = val

        else:
            gen = (key_1 for key_1 in counter_obj_n_gram.keys() if key_1[0:n-1] == key)
            sum_vals = 0
            for key_1 in gen:
                val = int(counter_obj_n_gram[key_1]) / int(counter_obj_n_1_gram[key])
                inner_dict[key_1[-1]] = val
                sum_vals += val

        lm_dict[key] = inner_dict.copy()

    return lm_dict

In [50]:
test_lm = lm(2, vocabulary, data_files['en_df'], False)

**Part 3**

Write a function *eval* that returns the perplexity of a model (dictionary) running over a given data file.

In [51]:
def eval(n, model, data_file):
    # n - the n-gram that you used to build your model (must be the same number)
    # model - the dictionary (model) to use for calculating perplexity
    # data_file - the tweets file that you wish to claculate a perplexity score for

    # read file
    if os.path.exists(data_file):
        text = tweets_to_text(data_file, n)
    else:
        text = data_file
    # Extract n length substrings
    n_gram = [text[i: i + n] for i in range(len(text) - n)]

    model_keys = model.keys()
    entropy = 0 
    for i_letter in n_gram:
        if i_letter[0:n-1] in model_keys: 
            i_letter_model = model[i_letter[0:n-1]]
            if i_letter[n-1] in i_letter_model.keys():
                second_letter_prob = i_letter_model[i_letter[n-1]]
                entropy += -np.log2(second_letter_prob)
            else:
                entropy += 0
        else:
            entropy += 0
    entropy = entropy/len(n_gram)
    perplexity_score = 2**(entropy)
    return perplexity_score

In [52]:
eval(2, test_lm, data_files['en_df'])

17.73587741775965

**Part 4**

Write a function *match* that creates a model for every relevant language, using a specific value of *n* and *add_one*. Then, calculate the perplexity of all possible pairs (e.g., en model applied on the data files en ,es, fr, in, it, nl, pt, tl; es model applied on the data files en, es...). This function should return a pandas DataFrame with columns [en ,es, fr, in, it, nl, pt, tl] and every row should be labeled with one of the languages. Then, the values are the relevant perplexity values.

In [53]:
def match(n, add_one, data_files):
    # n - the n-gram to use for creating n-gram models
    # add_one - use add_one smoothing or not
    result_dict = {}
    #vocabulary = preprocess(data_files)
    for i_language_model in languages_list:
        
        i_model = lm(n, vocabulary, data_files[i_language_model], add_one)
        result_dict[i_language_model] = {}

        for i_language_test in languages_list:
            i_language_model_i_score = eval(n, i_model, data_files[i_language_test])
            result_dict[i_language_model][i_language_test] = i_language_model_i_score
    perlexity_df = pd.DataFrame(result_dict)
    return perlexity_df  

**Part 5**

Run match with *n* values 1-4, once with add_one and once without, and print the 8 tables to this notebook, one after another.

In [54]:
 
def run_match(data_files):
    full_model_dict = {}
    # for n in range(2,3):

    for n in range(1,5):
        add_one = True
        perlexity_df = match(n, add_one, data_files)
        print(f'n = {n}, add_one = {add_one}')
        display(perlexity_df)

        add_one = False
        perlexity_df = match(n, add_one, data_files)
        print(f'n = {n}, add_one = {add_one}')
        display(perlexity_df)



# run the model generation

In [ ]:
%time
model_dict = run_match(data_files)


Wall time: 0 ns
n = 1, add_one = True


,en_df,es_df,fr_df,in_df,it_df,nl_df,pt_df,tl_df
en_df,38.630607,42.062439,41.630300,42.540889,41.498486,40.790634,42.542629,42.187752
es_df,40.802257,36.248897,39.694951,43.286563,38.918135,40.690781,37.529098,42.726271
fr_df,42.736129,40.896944,37.583780,46.985917,40.526005,41.867611,40.683119,47.511581
in_df,41.629066,43.776304,44.589019,37.553346,43.570273,41.669084,43.104367,39.224852
it_df,40.671132,40.132135,39.945922,43.329293,37.725009,40.997348,40.562942,42.745835
nl_df,39.718389,41.492501,40.919205,41.848385,41.084247,37.610331,41.625927,42.777840
pt_df,43.364948,39.777221,41.232575,45.617500,41.306291,42.919685,37.213175,45.019894
tl_df,44.850369,47.334181,49.266270,42.752191,46.498825,46.506471,47.208401,40.832146


n = 1, add_one = False


,en_df,es_df,fr_df,in_df,it_df,nl_df,pt_df,tl_df
en_df,38.578430,41.412538,40.878488,41.934916,40.635953,39.941618,41.730430,41.324155
es_df,38.309949,36.193704,39.057459,37.859935,38.449889,39.533533,37.082731,41.968109
fr_df,39.394444,38.854103,37.534046,44.793916,39.015645,41.087767,39.254733,45.022928
in_df,40.831635,42.844921,43.722744,37.497330,42.561710,40.746303,42.144046,38.403995
it_df,38.464632,38.563296,39.276591,41.336244,37.661463,39.965662,38.813462,40.745813
nl_df,39.289492,40.806654,40.372793,41.295066,40.484631,37.553020,40.871053,42.211687
pt_df,38.403357,36.071470,38.173631,39.073688,37.460498,39.864344,37.141313,43.158697
tl_df,44.130653,46.466069,48.493727,42.143952,45.622656,45.647622,46.216320,40.761592


**Part 6**

Each line in the file test.csv contains a sentence and the language it belongs to. Write a function that uses your language models to classify the correct language of each sentence.

Important note regarding the grading of this section: this is an open question, where a different solution will yield different accuracy scores. any solution that is not trivial (e.g. returning 'en' in all cases) will be excepted. We do reserve the right to give bonus points to exceptionally good/creative solutions.

In [ ]:
! ls nlp-course\lm-languages-data-new

In [ ]:
test_folder = f'nlp-course\lm-languages-data-new'
test_csv_files =  glob.glob(test_folder + '\\*.csv')
test_files =  {}
for i_file in test_csv_files:
    file_name_with_ending = os.path.basename(i_file)
    file_name = os.path.splitext(file_name_with_ending)[0]
    test_files[file_name + '_df'] = f'' + i_file


In [ ]:

def match_test(n, data_file_path, add_one):
    # n - the n-gram to use for creating n-gram models
    # add_one - use add_one smoothing or not
    #data_file_path = r"C:\MSC\NLP2\nlp-course\lm-languages-data-new\test.csv"
    senstences_list = pd.read_csv(data_file_path)['tweet_text'].to_list()

    lines = [] 
    result_dict = {}

    for i_language_model in languages_list:
        # i_model = model_dict[n][add_one][i_language_model]
        result_dict[i_language_model] = {}
        i_model = lm(n, vocabulary, data_files[i_language_model], add_one)

        for i_test_senstence_idx in range(senstences_list.__len__()):
            i_test_senstence = senstences_list[i_test_senstence_idx]
            i_sentence_model_i_score = eval(n, i_model, i_test_senstence)
            result_dict[i_language_model][i_test_senstence_idx] = i_sentence_model_i_score
    # print('summary for '+ i_language_model +' model perlexity score for each language:\n')
    perlexity_df = pd.DataFrame(result_dict)
    print(perlexity_df)
    perlexity_array = perlexity_df.to_numpy()
    language_match_index = np.argmin(perlexity_array, axis=1)
    language_match_list = reorder_list(languages_list, language_match_index)
    perlexity_df['predict'] = language_match_index
    perlexity_df['predict_language'] = language_match_list
    print(perlexity_df)

    #TODO
    return perlexity_df


def classify(n, data_file_path, add_one):
    # TODO
    match_dict  = match_test(n, data_file_path, add_one)
    return match_dict


In [ ]:
n = 2
test_path = test_folder + '\\test.csv'
clasification_result = classify(2, test_path, False)

In [ ]:
y_true = pd.read_csv(test_path).get('label').to_list()
y_true = list(map(lambda x: languages_list.index(x+'_df'),y_true))
y_pred = clasification_result['predict'].to_list()

**Part 7**

Calculate the F1 score of your output from part 6. (hint: you can use https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html). 


In [ ]:

def calc_f1(y_true,y_pred ):
    return np.round(f1_score(y_true, y_pred,average="micro"),3)
f_score_result = calc_f1(y_true,y_pred)
print('The F-score we acheive is ' + str(f_score_result)+'\n')


# **Good luck!**